# Spark Structured Streaming - Demo
## Fire alarm

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import io
from pyspark.sql.functions import *
import time
import json
import struct
import requests 

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.5,org.apache.kafka:kafka-clients:2.6.0 pyspark-shell'
                                    
spark = (SparkSession.builder 
    .master("local[*]")
    .appName("test")
    .getOrCreate()
        )

spark

set up the environment variables

In [2]:
smoke_topic = 'SmokeSensorEvent'
temperature_topic = 'TemperatureSensorEvent'
servers = "kafka:9092"

## Understanding spark-kafka integration
Let's treat first kafka as a bulk source

In [3]:
smoke_df = (spark
  .read
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("subscribe", smoke_topic)
  .option("startingOffsets", "earliest")
  .option("endingOffsets", "latest")
  .load())

In [4]:
smoke_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
smoke_df.show(5)

+-------+--------------------+----------------+---------+------+--------------------+-------------+
|    key|               value|           topic|partition|offset|           timestamp|timestampType|
+-------+--------------------+----------------+---------+------+--------------------+-------------+
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     0|2022-10-26 06:27:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     1|2022-10-26 06:27:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     2|2022-10-26 06:28:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     3|2022-10-26 06:28:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     4|2022-10-26 06:28:...|            0|
+-------+--------------------+----------------+---------+------+--------------------+-------------+



In [7]:
stringified_smoke_df = smoke_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
stringified_smoke_df.show(5,False)

+---+--------------------------------------------------+
|key|value                                             |
+---+--------------------------------------------------+
|S1 |{"sensor": "S1", "smoke": false, "ts": 1666765666}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1666765677}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1666765687}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1666765697}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1666765707}|
+---+--------------------------------------------------+



In [8]:
stringified_smoke_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [9]:
from pyspark.sql.types import *

smoke_schema = StructType([
    StructField("sensor", StringType(), True),
    StructField("smoke", BooleanType(), True),
    StructField("ts", TimestampType(), True)])

In [10]:
smoke_df = stringified_smoke_df.select(col("key").cast("string"),from_json(col("value"), smoke_schema).alias("value"))

In [11]:
smoke_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- sensor: string (nullable = true)
 |    |-- smoke: boolean (nullable = true)
 |    |-- ts: timestamp (nullable = true)



In [12]:
smoke_df.select("value.*").show(5)

+------+-----+-------------------+
|sensor|smoke|                 ts|
+------+-----+-------------------+
|    S1|false|2022-10-26 06:27:46|
|    S1|false|2022-10-26 06:27:57|
|    S1|false|2022-10-26 06:28:07|
|    S1|false|2022-10-26 06:28:17|
|    S1|false|2022-10-26 06:28:27|
+------+-----+-------------------+



## Let's explore Spark Structured Streaming by example
Please refer to [EPL fire allarm](https://github.com/emanueledellavalle/streaming-data-analytics/tree/main/codes/epl_firealarm) for the EPL version of the following queries.

### Let's create the streaming Data Frames using the data in the kafka smoke topic

In [13]:
raw_streaming_smoke_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", smoke_topic)
  .load())

In [14]:
raw_streaming_smoke_df.isStreaming

True

In [15]:
raw_streaming_smoke_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [16]:
smoke_sdf=(raw_streaming_smoke_df
                      .select(from_json(col("value").cast("string"), smoke_schema).alias("value"))
                      .select("value.*"))

In [17]:
smoke_sdf.printSchema()

root
 |-- sensor: string (nullable = true)
 |-- smoke: boolean (nullable = true)
 |-- ts: timestamp (nullable = true)



**NOTE**: it is not a DataFrame, you cannot directly execute an action on it. 

**The following cell *intetionally* gives an error**.

In [18]:
smoke_sdf.count()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
kafka

Queries with streaming sources must be registred and started with `writeStream.start()`

### Let's register and start a simple query

In [19]:
basic_query = (smoke_sdf
    .writeStream
    .format("memory") # this is for debug purpose only! DO NOT USE IN PRODUCTION
    .queryName("sinkTable")
    .start())

In [20]:
basic_query.lastProgress

{'id': 'f4a6e3c7-b360-4d50-a7de-ebba7db088b2',
 'runId': '9b980aa7-99ee-4ce7-adce-f66293930111',
 'name': 'sinkTable',
 'timestamp': '2022-10-26T06:30:55.040Z',
 'batchId': 0,
 'numInputRows': 5,
 'processedRowsPerSecond': 6.51890482398957,
 'durationMs': {'addBatch': 212,
  'getBatch': 22,
  'latestOffset': 258,
  'queryPlanning': 83,
  'triggerExecution': 766,
  'walCommit': 77},
 'stateOperators': [],
 'sources': [{'description': 'KafkaV2[Subscribe[SmokeSensorEvent]]',
   'startOffset': None,
   'endOffset': {'SmokeSensorEvent': {'0': 5}},
   'numInputRows': 5,
   'processedRowsPerSecond': 6.51890482398957}],
 'sink': {'description': 'MemorySink', 'numOutputRows': 5}}

run the following cell to see the most recent content of the sinkTable

In [21]:
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+-----+-------------------+
|sensor|smoke|                 ts|
+------+-----+-------------------+
|    S1|false|2022-10-26 06:28:27|
|    S1|false|2022-10-26 06:28:17|
|    S1|false|2022-10-26 06:28:07|
|    S1|false|2022-10-26 06:27:57|
|    S1|false|2022-10-26 06:27:46|
+------+-----+-------------------+



do not forget to stop queries that you are not using

In [22]:
basic_query.stop()

### Let's create the streaming Data Frames for the kafka temperature topic

In [23]:
temperarture_schema = StructType([
    StructField("sensor", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("ts", TimestampType(), True)])

raw_streaming_temperature_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", temperature_topic)
  .load())

temperature_sdf = (raw_streaming_temperature_df
                      .select(from_json(col("value").cast("string"), temperarture_schema).alias("value"))
                      .select("value.*"))

In [24]:
temperature_sdf.printSchema()

root
 |-- sensor: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- ts: timestamp (nullable = true)



## Q0 - Filter

The temperature events whose temperature is greater than 50 °C 

### the SQL style

In [25]:
# create a logic table on top of the streaming data frame
temperature_sdf.createTempView("TemperatureSensorEvent")

# write your query in SQL, register it and start it
q0 = (spark.sql("select * from TemperatureSensorEvent where temperature > 50")
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

let's ask for the execution plan, we will compare it with cells down with the one of the query in Data Frame style

In [26]:
q0.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@1cfd6f33
+- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).sensor AS sensor#289, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).temperature AS temperature#290, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).ts AS ts#291]
   +- Filter (from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).temperature > 50.0)
      +- *(1) Project [key#273, value#274, topic#275, partition#276, offset#277L, timestamp#278, timesta

In [27]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+-----------+---+
|sensor|temperature| ts|
+------+-----------+---+
+------+-----------+---+



if you are following carefully the instruction it should be empty because we are sending temperature aropund 20 °C.

Go back to the `temperature_sensor_simulator` notebook, stop the cell that is sending temperature around 20°C and run the one that sends temperature around 55°C

In [28]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+-----------------+-------------------+
|sensor|      temperature|                 ts|
+------+-----------------+-------------------+
|    S1|56.00886876914325|2022-10-26 06:31:46|
+------+-----------------+-------------------+



Now you should see results.

In [29]:
# clean up
q0.stop()
spark.catalog.dropTempView("TemperatureSensorEvent")

### The DataFrame style

In [30]:
q0bis = (temperature_sdf
                     .where("temperature > 50") # you can add anything that fits in a SQL where statemente 
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

let's ask for the explanation of the plan. Comparing with the one of the SQL style, you can see that there is no difference. This is expected because the [catalyst optimizer](https://databricks.com/glossary/catalyst-optimizer) created it out of our declarations (which are semantically equivalent)

In [31]:
q0bis.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@483f70da
+- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).sensor AS sensor#289, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).temperature AS temperature#290, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).ts AS ts#291]
   +- Filter (from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)).temperature > 50.0)
      +- *(1) Project [key#273, value#274, topic#275, partition#276, offset#277L, timestamp#278, timesta

In [32]:
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+-----------------+-------------------+
|sensor|      temperature|                 ts|
+------+-----------------+-------------------+
|    S1|54.22509249269572|2022-10-26 06:32:06|
|    S1|56.26551525029317|2022-10-26 06:31:56|
|    S1|56.00886876914325|2022-10-26 06:31:46|
+------+-----------------+-------------------+



In [33]:
q0bis.stop()

> NOTE: there was no need to
> * create a logic table on top of the streaming data frame with `temperature_sdf.createTempView("TemperatureSensorEvent")`
> * drop such a logic table with `spark.catalog.dropTempView("TemperatureSensorEvent")`

## Q1 - Avg

the average of all the temperature observation for each sensor up to the last event received

### the SQL sytyle

In [34]:
# create a logic table on top of the streaming data frame
temperature_sdf.createTempView("TemperatureSensorEvent") # this time we will not clean it up, because we use it in the next queries

**NOTE**: the following query gives *intentionally* an error

In [35]:
query_string = """
SELECT SENSOR, AVG(temperature) 
FROM TemperatureSensorEvent
GROUP BY SENSOR
"""

# write your query in SQL, register it and start it
q1 = (spark.sql(query_string)
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;;
Aggregate [SENSOR#289], [SENSOR#289, avg(temperature#290) AS avg(temperature)#432]
+- SubqueryAlias temperaturesensorevent
   +- Project [value#287.sensor AS sensor#289, value#287.temperature AS temperature#290, value#287.ts AS ts#291]
      +- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#274 as string), Some(Etc/UTC)) AS value#287]
         +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@4439d11c, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@54fe14de, org.apache.spark.sql.util.CaseInsensitiveStringMap@24f684c4, [key#273, value#274, topic#275, partition#276, offset#277L, timestamp#278, timestampType#279], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@14a73345,kafka,List(),None,List()

The **append output mode** (i.e., the default one) is not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark, we need to use the **complete output mode**.

In [36]:
query_string = """
SELECT SENSOR, AVG(temperature) 
FROM TemperatureSensorEvent
GROUP BY SENSOR
"""

# write your query in SQL, register it and start it
q1 = (spark.sql(query_string)
                     .writeStream
                     .format("memory")
                     .outputMode("complete") # <-- CHANGE HERE
                     .queryName("sinkTable")
                     .start())

In [39]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show() # without ORDER BY TS DESC because the result in the table is already only the most recent

+------+-----------------+
|SENSOR| avg(temperature)|
+------+-----------------+
|    S1|53.74661986795754|
+------+-----------------+



**NOTE**: if the cell above gives an empty result, wait 10 seconds and run it again. The very first excution may take time, expecially if you have already ingested many temperature events in kafka. Here we are querying the sink table and it may be empty because the first execution is still running.

In [40]:
# clean up
q1.stop()

### The DataFrame style

In [41]:
# write your query in SQL, register it and start it
q1bis = (temperature_sdf 
                     .groupBy("sensor")
                     .avg()
                     .writeStream
                     .format("memory")
                     .outputMode("complete") 
                     .queryName("sinkTable")
                     .start())

In [45]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show() # woithout ORDER BY TS DESC because the result in the table is already only the most recent

+------+-----------------+
|sensor| avg(temperature)|
+------+-----------------+
|    S1|53.76653609767246|
+------+-----------------+



In [46]:
# clean up
q1bis.stop()

## Q2 - Logical Sliding Window

The average temperature observed by each sensor in the last 4 seconds

MEMO: the average should change as soon as the receive a new event

**Not supported**

## Q3 - Logical Tumbling Window

The average temperature of the last 30 seconds every 30 seconds (was 4 seconds in EPL)

NOTE: this query is not possibile in the SQL style

In [47]:
q3 = (temperature_sdf
                  .groupBy(window("TS", "30 seconds"),"SENSOR")
                  .avg("TEMPERATURE")
                  .writeStream
                  .outputMode("complete")
                  .format("memory")
                  .queryName("sinkTable")
                  .start())

In [51]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(5,False) # NOTE: here we order by window instead of ordering by timestamp# window instead of timestamp, again

+------------------------------------------+------+------------------+
|window                                    |SENSOR|avg(TEMPERATURE)  |
+------------------------------------------+------+------------------+
|[2022-10-26 07:15:30, 2022-10-26 07:16:00]|S1    |55.32720389345427 |
|[2022-10-26 06:56:00, 2022-10-26 06:56:30]|S1    |54.75417122529101 |
|[2022-10-26 06:55:30, 2022-10-26 06:56:00]|S1    |55.09520890669898 |
|[2022-10-26 06:55:00, 2022-10-26 06:55:30]|S1    |55.310080691870475|
|[2022-10-26 06:54:30, 2022-10-26 06:55:00]|S1    |55.164726288446275|
+------------------------------------------+------+------------------+
only showing top 5 rows



In [52]:
q3.stop()

## Q4 - Physical Sliding Window

The moving average of the last 4 temperature events

**Not supported**

## Q5 - Physical Tumbling Window

The moving average of the last 4 temperature events every 4 events 

**Not supported**

## Q6 - Logical Hopping Window

The average temperature of the last 1 minute (was 4 seconds in EPL) every 30 seconds (was 2 seconds in EPL)

In [53]:
q6 = (temperature_sdf
      .groupBy(window("TS", "1 minutes", "30 seconds"),"SENSOR")
      .avg("TEMPERATURE")
      .writeStream
      .outputMode("complete")
      .format("memory")
      .queryName("sinkTable")
      .start())

In [56]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(6,False) # NOTE: here we order by window instead of ordering by timestamp

+------------------------------------------+------+------------------+
|window                                    |SENSOR|avg(TEMPERATURE)  |
+------------------------------------------+------+------------------+
|[2022-10-26 07:15:30, 2022-10-26 07:16:30]|S1    |56.68803759086834 |
|[2022-10-26 07:15:00, 2022-10-26 07:16:00]|S1    |56.68803759086834 |
|[2022-10-26 06:56:00, 2022-10-26 06:57:00]|S1    |54.75417122529101 |
|[2022-10-26 06:55:30, 2022-10-26 06:56:30]|S1    |54.95879383413579 |
|[2022-10-26 06:55:00, 2022-10-26 06:56:00]|S1    |55.202644799284734|
|[2022-10-26 06:54:30, 2022-10-26 06:55:30]|S1    |55.23740349015838 |
+------------------------------------------+------+------------------+
only showing top 6 rows



In [57]:
q6.stop()

## Q7 - Stream-to-Stream Join

In EPL, at this point we moved on to the pattern matching part required to satisfy the information need, i.e., "find every smoke event followed by a temperature event whose temperature is above 50 °C within 2 minutes."

Spark Structured Streaming does not support the EPL's operator `->` (that reads as *followed by*. We need to use a stream-to-stream join.

In [58]:
last_minute_smoke_events = (smoke_sdf
                .where("smoke = True")
                .withColumnRenamed("sensor","sensorSmoke")
                .withColumnRenamed("ts","tsSmoke")
               )

last_minute_high_temperature_events = (temperature_sdf
                .where("temperature > 50")
                .withColumnRenamed("sensor","sensorTemp")
                .withColumnRenamed("ts","tsTemp")
               )

Join with event-time constraints

In [59]:
join_sdf = (last_minute_smoke_events.join(
  last_minute_high_temperature_events, expr("""
    (sensorTemp == sensorSmoke) AND
    (tsTemp > tsSmoke ) AND
    (tsTemp < tsSmoke + interval 2 minute )
    """
    )))

In [60]:
q7 = (join_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

**IMPORTANT** To detect fire, run the appropriate cells in the data generators.

In [61]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsTemp DESC").show(20,False) # note, I change ts in tsTemp

+-----------+-----+-------+----------+-----------+------+
|sensorSmoke|smoke|tsSmoke|sensorTemp|temperature|tsTemp|
+-----------+-----+-------+----------+-----------+------+
+-----------+-----+-------+----------+-----------+------+



If you are following carefully the instructions the answer should be empty because we are sending temperature around 55°C, but there is no `smoke==true`, yet.

Go to the `smoke_sensor_simulator` notebook and start sending `smoke==true`.

In [69]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsTemp DESC").show(20,False) # note, I change ts in tsTemp

+-----------+-----+-------------------+----------+------------------+-------------------+
|sensorSmoke|smoke|tsSmoke            |sensorTemp|temperature       |tsTemp             |
+-----------+-----+-------------------+----------+------------------+-------------------+
|S1         |true |2022-10-26 07:17:14|S1        |54.407203182829065|2022-10-26 07:17:18|
+-----------+-----+-------------------+----------+------------------+-------------------+



let's have a look to the progresses

In [70]:
from IPython.display import clear_output
import json
while True:
    print(json.dumps(q7.lastProgress, indent=4))
    print(q7.status)
    time.sleep(1)
    clear_output(wait=True)
    

{
    "id": "5d195f3f-a119-46ab-9a63-01682dd7369a",
    "runId": "23d75bd5-ca0b-430f-8c63-d3669fcdb19b",
    "name": "sinkTable",
    "timestamp": "2022-10-26T07:19:30.340Z",
    "batchId": 12,
    "numInputRows": 2,
    "inputRowsPerSecond": 0.17725782150137376,
    "processedRowsPerSecond": 0.1388888888888889,
    "durationMs": {
        "addBatch": 14255,
        "getBatch": 0,
        "latestOffset": 2,
        "queryPlanning": 94,
        "triggerExecution": 14400,
        "walCommit": 19
    },
    "stateOperators": [
        {
            "numRowsTotal": 186,
            "numRowsUpdated": 2,
            "memoryUsedBytes": 382496,
            "customMetrics": {
                "loadedMapCacheHitCount": 4800,
                "loadedMapCacheMissCount": 0,
                "stateOnCurrentVersionSizeBytes": 67984
            }
        }
    ],
    "sources": [
        {
            "description": "KafkaV2[Subscribe[SmokeSensorEvent]]",
            "startOffset": {
                "Smo

KeyboardInterrupt: 

to interrupt the execution of the cell, prese the square icon in the bar or choose *interrupt kernel* from the *kernel* dropdown menu

#### Discussion

> This query is equivalent to the EPL pattern `every a = SmokeSensorEvent(smoke=true) -> every TemperatureSensorEvent(temperature > 50, sensor=a.sensor) where timer:within(1 min)`. 
>
> Do not expect the same performances! It is evaluated as a relational join. Spark Structured Streaming lacks the specilized data structure of Esper.
>
> **It does not tame the torrent effect**, but this is expected! 
>
> Spark Structured Streaming is a Data Stream Management System meant to tame *flow that you cannot stop*

Even id Q8 consumes Q7 results, we can stop Q7 because we only need the streaming Data Frame `join_sdf`. We do not need Q7 to write its result in the in memory table.

In [71]:
q7.stop()

## Q8 - Count FireEvent

we are very close to the solution of the running example, we "just" need to count the number of events generated by the previous query over an hopping window of 1 minutes that slides every 30 seconds (was a sliding window of 10 secondsin EPL). 

So let's count the results of Q7. 

**NOTE**: the following queries give *intentionally* errors

In [72]:
q8 = (join_sdf
            .groupBy(window("tsTemp", "1 minutes", "30 seconds"),"sensorTemp")
            .count()
            .writeStream
            .outputMode("complete")
            .format("memory")
            .queryName("sinkTable") 
            .start())

AnalysisException: Join between two streaming DataFrames/Datasets is not supported in Complete output mode, only in Append output mode;;
Join Inner, (((sensorTemp#16516 = sensorSmoke#16508) AND (tsTemp#16520 > tsSmoke#16512)) AND (tsTemp#16520 < cast(tsSmoke#16512 + 2 minutes as timestamp)))
:- Project [sensorSmoke#16508, smoke#216, ts#217 AS tsSmoke#16512]
:  +- Project [sensor#215 AS sensorSmoke#16508, smoke#216, ts#217]
:     +- Filter (smoke#216 = true)
:        +- Project [value#213.sensor AS sensor#215, value#213.smoke AS smoke#216, value#213.ts AS ts#217]
:           +- Project [from_json(StructField(sensor,StringType,true), StructField(smoke,BooleanType,true), StructField(ts,TimestampType,true), cast(value#200 as string), Some(Etc/UTC)) AS value#213]
:              +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@a96be10, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@3f675c30, org.apache.spark.sql.util.CaseInsensitiveStringMap@7098e

Indeed, join between two streaming DataFrames/Datasets is not supported in Complete output mode, only in Append output mode. So let's try to use append mode.

In [73]:
q8 = (join_sdf
            .groupBy(window("tsTemp", "1 minutes", "30 seconds"),"sensorTemp")
            .count()
            .writeStream
            .outputMode("append") # <-- CHANGE HERE
            .format("memory")
            .queryName("sinkTable") 
            .start())

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;;
Aggregate [window#87092, sensorTemp#16516], [window#87092 AS window#87083, sensorTemp#16516, count(1) AS count#87091L]
+- Filter ((tsTemp#16520 >= window#87092.start) AND (tsTemp#16520 < window#87092.end))
   +- Expand [ArrayBuffer(named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(tsTemp#16520, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) as double) = (cast((precisetimestampconversion(tsTemp#16520, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) THEN (CEIL((cast((precisetimestampconversion(tsTemp#16520, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) + cast(1 as bigint)) ELSE CEIL((cast((precisetimestampconversion(tsTemp#16520, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) END + cast(0 as bigint)) -

Append output mode is not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark. Indeed, **the streaming join can create out of orders**.

Let's add a watermark, then.

In [74]:
q8 = (join_sdf
            .withWatermark(delayThreshold="2 minutes",eventTime="tsTemp") # <-- CHANGE HERE
            .groupBy(window("tsTemp", "1 minutes", "30 seconds"),"sensorTemp")
            .count()
            .writeStream
            .outputMode("append") 
            .format("memory")
            .queryName("sinkTable") 
            .start())

NOTE: 2 minutes is maximum delay that the join can cause given the way we declared it. The temperature and the smoke event cannot be more than 2 minutes apart.

In [81]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(5,False)

+------------------------------------------+----------+-----+
|window                                    |sensorTemp|count|
+------------------------------------------+----------+-----+
|[2022-10-26 07:17:30, 2022-10-26 07:18:30]|S1        |33   |
|[2022-10-26 07:17:00, 2022-10-26 07:18:00]|S1        |15   |
|[2022-10-26 07:16:30, 2022-10-26 07:17:30]|S1        |3    |
+------------------------------------------+----------+-----+



In [82]:
q8.stop()